In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Revision_Paper1/

Mounted at /content/drive
/content/drive/My Drive/Revision_Paper1


In [2]:
import pandas as pd
from glob import glob
import os
from scipy.io import loadmat
import numpy as np

In [3]:
file_path1 = '/content/drive/My Drive/Revision_Paper1/beta_wa2.mat'
data1 = loadmat(file_path1)["reshape"]
file_path2 = '/content/drive/My Drive/Revision_Paper1/beta_wb2.mat'
data2 = loadmat(file_path2)["reshape"]
file_path3 = '/content/drive/My Drive/Revision_Paper1/beta_wc2.mat'
data3 = loadmat(file_path3)["reshape"]
file_path4 = '/content/drive/My Drive/Revision_Paper1/beta_wd2.mat'
data4 = loadmat(file_path4)["reshape"]

In [4]:
import numpy as np

def standardize_data(*datasets):
    standardized = []
    for data in datasets:
        mean = data.mean(axis=0)
        std = data.std(axis=0)
        standardized_data = (data - mean) / std
        standardized.append(standardized_data)
    return standardized


In [6]:
d1, d2, d3, d4 = standardize_data(data1, data2, data3, data4)


In [13]:
data_array = np.concatenate([d1, d2, d3, d4])
n = data_array.shape[0] // 4

labels = np.concatenate((
    np.zeros(n),
    np.ones(n),
    np.full(n, 2),
    np.full(n, 3)
))


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense, TimeDistributed, Reshape
from keras.layers import GlobalAveragePooling1D, Conv2D, GRU, AveragePooling2D, DepthwiseConv1D, SeparableConv1D, LSTM, MaxPool1D, MaxPooling2D, AveragePooling2D,Conv1D
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input,  concatenate, add
from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Embedding, Layer
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ModelCheckpoint

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracy = []
fold_loss = []
all_y_true = []
all_y_pred = []

# Cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(x, y)):
    print(f"Training fold {fold + 1}...")

    # Split data for current fold
    x_train_fold, x_val_fold = x[train_index], x[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Define the model architecture within the loop for a fresh model each fold
    input_layer = Input(shape=(656, 21, 1))
    conv1 = Conv2D(15, (21, 1), activation='relu', padding='same')(input_layer)
    conv = Conv2D(15, (1, 20), activation='relu', padding='same')(conv1)
    conv1a = Conv2D(16, (1, 1), activation='relu', padding='same')(conv)
    max1 = MaxPooling2D((2, 2), padding='same')(conv1a)
    conv3a = Conv2D(32, (3, 3), activation='relu', padding='same')(conv)
    max2 = MaxPooling2D((2, 2), padding='same')(conv3a)
    conv5a = Conv2D(64, (5, 5), activation='relu', padding='same')(conv)
    max3 = MaxPooling2D((2, 2), padding='same')(conv5a)
    concat = concatenate([max1, max2, max3], axis=-1)
    maxpool = MaxPooling2D((3, 3), padding='valid')(concat)
    reshaped = Reshape((maxpool.shape[1] * maxpool.shape[2], maxpool.shape[3]))(maxpool)

    # Transformer encoder
    transformer_layer = transformer_encoder(inputs=reshaped, head_size=128, num_heads=3, ff_dim=128, dropout=0.5)
    output_layer = Flatten()(transformer_layer)
    output_layer = Dense(4, activation='softmax')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        x_train_fold, y_train_fold,
        validation_data=(x_val_fold, y_val_fold),
        epochs=50,
        batch_size=16,
        verbose=1
    )
    val_loss, val_accuracy = model.evaluate(x_val_fold, y_val_fold, verbose=0)
    fold_accuracy.append(val_accuracy)
    fold_loss.append(val_loss)

    # Predictions
    y_pred_fold = np.argmax(model.predict(x_val_fold), axis=1)
    all_y_true.extend(y_val_fold)
    all_y_pred.extend(y_pred_fold)

# Calculate overall metrics
conf_matrix = confusion_matrix(all_y_true, all_y_pred)
precision = precision_score(all_y_true, all_y_pred, average='weighted')
recall = recall_score(all_y_true, all_y_pred, average='weighted')
f1 = f1_score(all_y_true, all_y_pred, average='weighted')

# Output results
print(f"\nAverage validation accuracy across folds: {np.mean(fold_accuracy)}")
print(f"Average validation loss across folds: {np.mean(fold_loss)}")
print("\nConfusion Matrix:")
print(conf_matrix)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


In [ ]:
import matplotlib.pyplot as plt

train_accuracy_per_fold = []
val_accuracy_per_fold = []

for history in history_list:
    train_accuracy_per_fold.append(history['accuracy'])
    val_accuracy_per_fold.append(history['val_accuracy'])

avg_train_accuracy = np.mean(train_accuracy_per_fold, axis=0)
avg_val_accuracy = np.mean(val_accuracy_per_fold, axis=0)

# Plot the averaged accuracies
plt.figure(figsize=(10, 5))
plt.ylim(0,1)
plt.yticks(np.arange(0, 1.1,0.1))

# Plot training accuracy
plt.plot(avg_train_accuracy, label='Average Train Accuracy', linestyle='-', color='b')

# Plot validation accuracy
plt.plot(avg_val_accuracy, label='Average Val Accuracy', linestyle='--', color='r')

plt.title('Average Accuracy Across Folds_Beta MA_2 class')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
